In [1]:
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
import keras.applications
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras import backend as K
import os
from sklearn.metrics import confusion_matrix
import numpy as np
import cv2

Using TensorFlow backend.


In [2]:
from google.colab import drive
# Dataset is stored at Google Drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
img_shape = (150, 150, 3)
tb_log_dir = 'logs'
num_epochs = 2
num_classes = 6
train_directory = '/content/drive/My Drive/intel-image-classification/seg_train'
test_directory = '/content/drive/My Drive/intel-image-classification/seg_test'

In [0]:
def clear_folder(folder):
    if os.path.exists(folder):
        for the_file in os.listdir(folder):
            file_path = os.path.join(folder, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                # elif os.path.isdir(file_path): shutil.rmtree(file_path)
            except Exception as e:
                print(e)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [5]:

import math
import random
from imgaug import augmenters as iaa



def preprocess_image(image_path):
    image = cv2.imread(image_path) #Reading the image (OpenCV) in BGR
    image = cv2.resize(image,(150,150)) #Resize the image
    image = image / 255
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # convert to grayscale
    return image

def augment_image(images):
    seq = iaa.Sequential([
      iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen), cropped pixels are set black
      iaa.Fliplr(0.5), # horizontally flip 50% of the images
      iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
    ])
    images_aug = seq(images=images)
    return images_aug


def load_train_val_images(directory, classes, val_split):    
    
    imagefilepaths = []
    imagefilepaths_train = []
    imagefilepaths_val = []
    y = []

    assert(val_split) > 0.0
    
    # Get file paths
    for labels in os.listdir(directory): 

        if os.path.isdir(os.path.join(directory,labels)): # Only consider folders
          
          label = classes[labels]
          for image_file in os.listdir(directory+ '/' + labels):
              imagefilepaths.append(directory+ '/' + labels+'/'+image_file)
              y.append(label)

    # Shuffle before loading images is more efficient
    zipped = list(zip(imagefilepaths, y))
    random.shuffle(zipped)
    imagefilepaths, y = zip(*zipped)

    # Split into test and validation split
    split_idx = math.floor(val_split * len(imagefilepaths))
    imagefilepaths_val = imagefilepaths[:split_idx]
    valy = np.array(y[:split_idx]) 
    imagefilepaths_train = imagefilepaths[split_idx:]
    trainy = np.array(y[split_idx:]) 

    # RAM reasons
    imagefilepaths = None
    y = None
    trainx = []
    valx = []
  
    
    for i, image_path in enumerate(imagefilepaths_train):
        image = preprocess_image(image_path)
        trainx.append(image)
        if i % 1000 == 999 :
            print('Preprocessing train image ' + str(i) + ' of ' + str(len(imagefilepaths_train)))

    for i,image_path in enumerate(imagefilepaths_val):
        image = preprocess_image(image_path)
        valx.append(image)
        if i % 1000 == 999:
            print('Preprocessing val image ' + str(i) + ' of ' + str(len(imagefilepaths_val)))

    #TODO: add image augmentation
    np.array(trainx)
    trainx = augment_image(trainx)

    return trainx, trainy, np.array(valx), valy

def load_test_images(directory, classes):
    testx = []
    testy = [] 


    for labels in os.listdir(directory):  

      if os.path.isdir(os.path.join(directory,labels)): # Only consider folders
        label = classes[labels]
        print(labels)

        for image_file in os.listdir(directory+ '/' + labels): #Extracting the file name of the image from Class Label folder
            image = preprocess_image(directory+ '/' + labels + '/' + image_file)
            testx.append(image)
            testy.append(label)
  

    return testx, testy

classes = {2:'glacier', 4:'sea', 0:'buildings', 1:'forest', 5:'street', 3:'mountain'}
reverse_classes = {'glacier':2, 'sea':4, 'buildings':0, 'forest':1, 'street':5, 'mountain':3}

xtrain, ytrain, xval, yval = load_train_val_images(train_directory, reverse_classes, 0.25)
np.save(train_directory+ '/trainxdata2.npy', xtrain)
np.save(train_directory+ '/trainydata2.npy', ytrain)
np.save(train_directory+ '/valxdata2.npy', xval)
np.save(train_directory+ '/valydata2.npy', yval)
xtest, ytest = load_test_images(test_directory, reverse_classes)
np.save(test_directory+ '/testxdata2.npy', xtest)
np.save(test_directory+ '/testydata2.npy', ytest)


Preprocessing train image 999 of 10542
Preprocessing train image 1999 of 10542
Preprocessing train image 2999 of 10542
Preprocessing train image 3999 of 10542
Preprocessing train image 4999 of 10542
Preprocessing train image 5999 of 10542
Preprocessing train image 6999 of 10542
Preprocessing train image 7999 of 10542
Preprocessing train image 8999 of 10542
Preprocessing train image 9999 of 10542
Preprocessing val image 999 of 3514
Preprocessing val image 1999 of 3514
Preprocessing val image 2999 of 3514
sea
mountain
street
glacier
forest
buildings


In [0]:
def build_model():

    basemodel = keras.applications.vgg16.VGG16(
        input_shape=img_shape,
        include_top=False,
        weights='imagenet',
        pooling='avg')

    #for layer in basemodel.layers:
    #    layer.trainable = False

    basemodel.summary()
  
    model = Sequential()
    model.add(basemodel)
    '''
    model.add(Conv2D(32, kernel_size=(3, 3), input_shape =img_shape, padding='same', activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size = (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))

    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    '''
    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1028, activation= 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()

    return model

In [0]:
seed = 42
batch_size = 32

xtrain = np.load(train_directory+ '/trainxdata2.npy') 
ytrain = np.load(train_directory+ '/trainydata2.npy')
xval = np.load(train_directory+ '/valxdata2.npy')
yval = np.load(train_directory+ '/valydata2.npy')
xtest = np.load(test_directory+ '/testxdata2.npy')
ytest = np.load(test_directory+ '/testydata2.npy')

print(xtrain.shape)

ytrain = keras.utils.to_categorical(ytrain)
yval = keras.utils.to_categorical(yval)
ytest = keras.utils.to_categorical(ytest)

'''# different pipeline needed, because reading images from google drive is really really slow
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    data_format='channels_last',
    validation_split=0.25,
    #shear_range=0.2,
    #zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_data = train_datagen.flow_from_directory(
    train_directory,
    target_size=img_shape[:-1], # only height and width
    batch_size=batch_size,
    seed=seed,
    class_mode='categorical',
    shuffle=True,
    color_mode='rgb')

test_data = test_datagen.flow_from_directory(
    test_directory,
    target_size=img_shape[:-1], # only height and width
    class_mode='categorical',
    color_mode='rgb')

validation_data = train_datagen.flow_from_directory(
    train_directory, # same directory as training data
    target_size=img_shape[:-1],
    class_mode='categorical',
    batch_size=batch_size,
    subset='validation') # set as validation data'''



clear_folder(tb_log_dir)
tensorboard = TensorBoard(log_dir=tb_log_dir)
checkpoint = ModelCheckpoint(train_directory+ 'weights.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model = build_model()
optimizer = optimizers.RMSprop(lr=1e-4)
# Labels are one hot encoded
model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics=["accuracy"])
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
print(model.summary)

model.fit(xtrain, ytrain, epochs=15, batch_size=32, validation_data =(xval, yval), shuffle=True, callbacks=[tensorboard, checkpoint])
prediction = model.predict(xtest)


#class_labels = list(reverse_classes.keys())   

#confusion = metrics.classification_report(np.argmax(ytest, axis = 1), prediction, target_names=class_labels)

(10542, 150, 150, 3)
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 